In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import lgamma, log
from scipy.special import gammaln

### Task1 (a)

In [2]:
# Read the dataset
file_path = "prelim2025data.txt"
df = pd.read_csv(file_path, comment='#', delim_whitespace=True, names=['b1', 'b2', 'r1', 'r2'])

/var/folders/tl/v6npptpd03n4cv7pcfzbqs6h0000gn/T/ipykernel_54380/3792862751.py:3: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(file_path, comment='#', delim_whitespace=True, names=['b1', 'b2', 'r1', 'r2'])


In [3]:
df.head()

,b1,b2,r1,r2
0,6,2,2,4
1,4,6,3,6
2,4,6,3,5
3,6,6,5,5
4,1,5,6,2


In [4]:
df.describe()

,b1,b2,r1,r2
count,28.000000,28.000000,28.000000,28.000000
mean,3.357143,3.964286,3.285714,4.107143
std,1.704336,1.477790,1.487158,1.499118
min,1.000000,2.000000,1.000000,2.000000
25%,2.000000,3.000000,2.000000,3.000000
50%,3.000000,4.000000,3.000000,4.000000
75%,5.000000,5.000000,4.250000,5.250000
max,6.000000,6.000000,6.000000,6.000000


In [13]:
# Count faces for each die (1-6)
face_counts = {}
for die in ['b1', 'b2', 'r1', 'r2']:
    counts = df[die].value_counts()
    counts = counts.reindex(range(1,7), fill_value=0)
    face_counts[die] = counts

face_counts_df = pd.DataFrame(face_counts)
print(face_counts_df)

   b1  b2  r1  r2
1   5   0   4   0
2   5   6   4   6
3   5   6   9   4
4   5   5   4   6
5   4   5   5   5
6   4   6   2   7


### Task1 (b)

The expected value of a fair six-sided die is calculated as:

$
E[X] = \sum_{i=1}^{6} i \cdot \frac{1}{6} = \frac{1 + 2 + 3 + 4 + 5 + 6}{6} = \frac{21}{6} = 3.5
$

So, the average outcome is 3.5.

For blue1 die, it shows mean of 3.35, closed to 3.5. Min and Max are respectively 1 and 6. Based on the balanced distribution and relatively normal spread, it is likely fair.

For blue2 die, it shows mean of 3.96, larger than 3.5. There is no 1 in this die due to min of 2. This is relatively unusual. Also, 25%, 50% and 75% tend to be higher. So I think this die may be tampered.

For red1 die, it shows mean of 3.29, also closed to 3.5. The range is also from 1 to 6 and no obviously skewed. So this is likely to be fair.

For red2 die, it shows mean of 4.11, higher than 3.5. Min is 2, showing no 1 in many rolls. It's weirld. The die shows high numbers in 25%, 50% and 75%. So, this die also may be tampered.

### Task2 (a) and (b) Please see attached paper handwriting.

### Task2 (c)

In [8]:
# Log multinomial coefficient
def log_multinomial_coef(counts):
    N = sum(counts)
    return gammaln(N + 1) - sum([gammaln(c + 1) for c in counts])


# H1: Fair die
def log_ml_h1(counts):
    term1 = log_multinomial_coef(counts)
    term2 = sum([c * log(1/6) for c in counts])
    return term1 + term2

# H2: Dirichlet-multinomial with uniform prior
def log_ml_h2(counts):
    alpha = [1] * 6
    sum_alpha = sum(alpha)
    sum_counts = sum(counts)

    term0 = log_multinomial_coef(counts)
    term1 = gammaln(sum_alpha) - gammaln(sum_counts + sum_alpha)
    term2 = sum([gammaln(n_i + 1) for n_i in counts])
    return term0 + term1 + term2

# H3: Tampered die (two sixes, no ones)
def log_ml_h3(counts):
    theta = [0, 1/6, 1/6, 1/6, 1/6, 2/6]
    if counts[0] > 0:
        return -float('inf')
    term2 = sum([c * log(theta[i]) for i, c in enumerate(counts) if theta[i] > 0])
    return log_multinomial_coef(counts) + term2

# H4: Tampered die (two fives, no ones)
def log_ml_h4(counts):
    theta = [0, 1/6, 1/6, 1/6, 2/6, 1/6]
    if counts[0] > 0:
        return -float('inf')
    term2 = sum([c * log(theta[i]) for i, c in enumerate(counts) if theta[i] > 0])
    return log_multinomial_coef(counts) + term2

# Evaluate each die
results = []
for die in ['b1', 'b2', 'r1', 'r2']:
    counts = [df[die].value_counts().get(i, 0) for i in range(1, 7)]

    print(f"{die.upper()} counts = {counts}, H2 = {log_ml_h2(counts)}")

    # Compute log marginal likelihoods
    logml_h1 = log_ml_h1(counts)
    logml_h2 = log_ml_h2(counts)
    logml_h3 = log_ml_h3(counts)
    logml_h4 = log_ml_h4(counts)

    logmls = {
        'Fair (H1)': logml_h1,
        'Dirichlet (H2)': logml_h2,
        'Two sixes, no ones (H3)': logml_h3,
        'Two fives, no ones (H4)': logml_h4
    }
    best_hyp = max(logmls, key=logmls.get)

    results.append({
        'Die': die.upper(),
        'Fair (H1)': f"{logml_h1:.2f}",
        'Dirichlet (H2)': f"{logml_h2:.2f}",
        'Two sixes, no ones (H3)': f"{logml_h3:.2f}",
        'Two fives, no ones (H4)': f"{logml_h4:.2f}",
        'Verdict': best_hyp
    })

# Display results
results_df = pd.DataFrame(results)
print("\nLog Marginal Likelihoods for each die and hypothesis:")
print(results_df.to_string(index=False))

B1 counts = [np.int64(5), np.int64(5), np.int64(5), np.int64(5), np.int64(4), np.int64(4)], H2 = -12.377232137617941
B2 counts = [0, np.int64(6), np.int64(6), np.int64(5), np.int64(5), np.int64(6)], H2 = -12.377232137617938
R1 counts = [np.int64(4), np.int64(4), np.int64(9), np.int64(4), np.int64(5), np.int64(2)], H2 = -12.377232137617938
R2 counts = [0, np.int64(6), np.int64(4), np.int64(6), np.int64(5), np.int64(7)], H2 = -12.377232137617938

Log Marginal Likelihoods for each die and hypothesis:
Die Fair (H1) Dirichlet (H2) Two sixes, no ones (H3) Two fives, no ones (H4)                 Verdict
 B1     -7.79         -12.38                    -inf                    -inf               Fair (H1)
 B2    -11.59         -12.38                   -7.43                   -8.13 Two sixes, no ones (H3)
 R1    -10.10         -12.38                    -inf                    -inf               Fair (H1)
 R2    -11.93         -12.38                   -7.08                   -8.46 Two sixes, no on

### Task2 (d) Discussion

Blue1 and red1 dice are seen as fair dice (H1) since they both have ones so eliminate H3 and H4. Also, -7.79 and -10.10 (H1) are higher than -12.38 (H2) so they are fair.

Blue2 and red2 dice are classified as tampered for H3 (two sixes, no ones) since both show zero ones, making H3/H4 viable. B2: H3 = -7.43 vs H4 = -8.13 and R2: H3 = -7.08 vs H4 = -8.46 show strong preference for H3 over H4. Also, Higher sixes counts (B2:6, R2:7) match H3's θ₆ = 2/6.